In [1]:
import findspark
findspark.init()

In [2]:
findspark.find()

'C:\\spark\\spark-2.4.6-bin-hadoop2.7'

In [3]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [4]:
conf = pyspark.SparkConf().setAppName('weather').setAll([('spark.maximizeResourceAllocation', 'true')])
sc = pyspark.SparkContext(conf=conf)

In [5]:
sc._jsc.hadoopConfiguration().setInt("dfs.blocksize", 128*1024*1024)
sc._jsc.hadoopConfiguration().setInt("parquet.block.size", 1024*1024*1024)
spark = SparkSession(sc)

In [17]:
df = spark.read.option("header", "true").csv("*.csv")

In [18]:
df = df.dropDuplicates()

In [19]:
df = df.withColumn("ScreenTemperature", df["ScreenTemperature"].cast("double"))

In [12]:
df.describe("ForecastSiteCode","ObservationTime", "ObservationDate", "ScreenTemperature", "Region", "Country").show()

+-------+------------------+------------------+-------------------+------------------+--------------------+-------+
|summary|  ForecastSiteCode|   ObservationTime|    ObservationDate| ScreenTemperature|              Region|Country|
+-------+------------------+------------------+-------------------+------------------+--------------------+-------+
|  count|            194559|            194559|             194559|            194559|              194559| 166819|
|   mean| 4740.680703539801|11.516573378769422|               null|3.3324055941899386|                null|   null|
| stddev|11120.543403963862| 6.932137870487246|               null|13.349654981147506|                null|   null|
|    min|              3002|                 0|2016-02-01T00:00:00|             -99.0|Central Tayside &...|ENGLAND|
|    max|             99214|                 9|2016-03-31T00:00:00|              15.8|  Yorkshire & Humber|  WALES|
+-------+------------------+------------------+-------------------+-----

In [20]:
df = df.withColumn("ObservationDate", df["ObservationDate"].cast("date"))

In [21]:
from pyspark.sql import Window
w = Window.partitionBy(["ObservationDate", "Region"])

In [26]:
import pyspark.sql.functions as f
df = df.withColumn('maxScreenTemperature', f.max('ScreenTemperature').over(w))\
       .where(f.col('ScreenTemperature') == f.col('maxScreenTemperature'))\
       .drop('maxScreenTemperature')

In [27]:
df.head(5)

[Row(ForecastSiteCode='3023', ObservationTime='15', ObservationDate=datetime.date(2016, 2, 23), WindDirection='13', WindSpeed='6', WindGust=None, Visibility='30000', ScreenTemperature=8.1, Pressure=None, SignificantWeatherCode='7', SiteName='SOUTH UIST RANGE (3023)', Latitude='57.3580', Longitude='-7.3970', Region='Highland & Eilean Siar', Country='SCOTLAND'),
 Row(ForecastSiteCode='3148', ObservationTime='16', ObservationDate=datetime.date(2016, 3, 17), WindDirection='12', WindSpeed='5', WindGust=None, Visibility=None, ScreenTemperature=12.6, Pressure=None, SignificantWeatherCode='-99', SiteName='GLEN OGLE (3148)', Latitude='56.4230', Longitude='-4.3200', Region='Central Tayside & Fife', Country='SCOTLAND'),
 Row(ForecastSiteCode='3017', ObservationTime='10', ObservationDate=datetime.date(2016, 3, 30), WindDirection='13', WindSpeed='8', WindGust=None, Visibility='45000', ScreenTemperature=7.2, Pressure='1000', SignificantWeatherCode='7', SiteName='KIRKWALL (3017)', Latitude='58.9540',

In [28]:
from pyspark.sql.functions import year, month, dayofmonth

In [29]:
df= df.withColumn("partitionYear", year("ObservationDate")).withColumn("partitionMonth", month("ObservationDate")).withColumn("partitionDay", dayofmonth("ObservationDate"))

In [30]:
df.count()

1143

In [31]:
df.repartition(1).write.mode('overwrite').partitionBy("partitionYear", "partitionMonth", "partitionDay").parquet("./result")